In [65]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Input, InputLayer, Dropout, Dense, Flatten, Embedding, Add, Concatenate
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

import import_ipynb
import metadata_options
import models_nn

In [41]:
df = pd.read_csv('./data/kokil dec 6 reprepare/conf_pc_worker_sem.csv')
df = df.dropna()

In [42]:
df['Input.deception_quadrant'] = df["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
df_deception = df[df['Input.deception_quadrant'] == 1]
df_no_deception = df[df['Input.deception_quadrant'] == 0]

In [43]:
indices_d = range(df_deception.shape[0])
indices_no_d = range(df_no_deception.shape[0])
df_deception_train, df_deception_test, df_deception_indices_train, df_deception_indices_test = train_test_split(df_deception, indices_d, test_size=0.2)
df_no_deception_train, df_no_deception_test, df_no_deception_indices_train, df_no_deception_indices_test = train_test_split(df_no_deception, indices_no_d, test_size=0.2)

In [44]:
y_deception_train_rapport = df_deception_train['Answer.3rapport.yes_label'].tolist()
y_deception_test_rapport = df_deception_test['Answer.3rapport.yes_label'].tolist()

y_deception_train_deception = df_deception_train['Input.deception_quadrant'].tolist()
y_deception_test_deception = df_deception_test['Input.deception_quadrant'].tolist()


y_no_deception_train_rapport = df_deception_train['Answer.3rapport.yes_label'].tolist()
y_deception_test_rapport = df_deception_test['Answer.3rapport.yes_label'].tolist()

y_no_deception_train_deception = df_no_deception_train['Input.deception_quadrant'].tolist()
y_no_deception_test_deception = df_no_deception_test['Input.deception_quadrant'].tolist()

In [61]:
X_deception_train_col = df_deception_train['Input.full_text']
X_deception_test_col = df_deception_test['Input.full_text']

X_no_deception_train_col = df_no_deception_train['Input.full_text']
X_no_deception_test_col = df_no_deception_test['Input.full_text']

In [46]:
le = LabelEncoder()

y_deception_train_rapport = le.fit_transform(y_deception_train_rapport)
y_deception_train_rapport = y_deception_train_rapport.reshape(-1,1)

y_deception_test_rapport = le.fit_transform(y_deception_test_rapport)
y_deception_test_rapport = y_deception_test_rapport.reshape(-1,1)

y_deception_train_deception = le.fit_transform(y_deception_train_deception)
y_deception_train_deception = y_deception_train_deception.reshape(-1,1)

y_deception_test_deception = le.fit_transform(y_deception_test_deception)
y_deception_test_deception = y_deception_test_deception.reshape(-1,1)

y_no_deception_train_rapport = le.fit_transform(y_no_deception_train_rapport)
y_no_deception_train_rapport = y_no_deception_train_rapport.reshape(-1,1)

y_deception_test_rapport = le.fit_transform(y_deception_test_rapport)
y_deception_test_rapport = y_deception_test_rapport.reshape(-1,1)

y_no_deception_train_deception = le.fit_transform(y_no_deception_train_deception)
y_no_deception_train_deception = y_no_deception_train_deception.reshape(-1,1)

y_no_deception_test_deception = le.fit_transform(y_no_deception_test_deception)
y_no_deception_test_deception = y_no_deception_test_deception.reshape(-1,1)

C:\Users\adria\.conda\envs\tfgpu\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [62]:
max_words = 1000
max_len = 220

tok = Tokenizer(num_words=max_words)

tok.fit_on_texts(X_deception_train_col)
X_deception_train_sequences = tok.texts_to_sequences(X_deception_train_col)
X_deception_train = pad_sequences(X_deception_train_sequences, maxlen=max_len)

X_deception_test_sequences = tok.texts_to_sequences(X_deception_test_col)
X_deception_test = pad_sequences(X_deception_test_sequences, maxlen=max_len)

tok2 = Tokenizer(num_words=max_words)
tok2.fit_on_texts(X_no_deception_train_col)
X_no_deception_train_sequences = tok2.texts_to_sequences(X_no_deception_train_col)
X_no_deception_train = pad_sequences(X_no_deception_train_sequences, maxlen=max_len)

X_no_deception_test_sequences = tok2.texts_to_sequences(X_no_deception_test_col)
X_no_deception_test = pad_sequences(X_no_deception_test_sequences, maxlen=max_len)

In [58]:
model_name = 'cnn'
models_nn.MODEL_NAME = model_name

In [55]:
X_deception_train.shape, y_deception_train_deception.shape, X_deception_test.shape, y_deception_test_deception.shape

((8646, 220), (8646, 1), (2162, 220), (2162, 1))

In [63]:
deception_model = models_nn.create_nn_model()
deception_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, 
                                                                              models_nn.recall_m, models_nn.precision_m])
deception_model.fit(X_deception_train, y_deception_train_deception,
                  batch_size=128,
                  epochs=15,
                  validation_data=(X_deception_test, y_deception_test_deception), 
                  callbacks=[models_nn.early_stop])

Epoch 1/15
68/68 [==============================] - 1s 11ms/step - loss: 0.0811 - accuracy: 0.9607 - f1_m: 0.0000e+00 - recall_m: 0.0000e+00 - precision_m: 0.0000e+00 - val_loss: 3.4301e-06 - val_accuracy: 1.0000 - val_f1_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00
Epoch 2/15
68/68 [==============================] - 1s 8ms/step - loss: 1.7687e-06 - accuracy: 1.0000 - f1_m: 0.0000e+00 - recall_m: 0.0000e+00 - precision_m: 0.0000e+00 - val_loss: 6.3690e-08 - val_accuracy: 1.0000 - val_f1_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00


In [64]:
deception_model.fit(X_no_deception_train, y_no_deception_train_deception,
                  batch_size=128,
                  epochs=15,
                  validation_data=(X_no_deception_test, y_no_deception_test_deception), 
                  callbacks=[models_nn.early_stop])

Epoch 1/15
4/4 [==============================] - 1s 270ms/step - loss: 8.0279e-08 - accuracy: 1.0000 - f1_m: 0.0000e+00 - recall_m: 0.0000e+00 - precision_m: 0.0000e+00 - val_loss: 5.0784e-08 - val_accuracy: 1.0000 - val_f1_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00
Epoch 2/15
4/4 [==============================] - 0s 15ms/step - loss: 6.4402e-08 - accuracy: 1.0000 - f1_m: 0.0000e+00 - recall_m: 0.0000e+00 - precision_m: 0.0000e+00 - val_loss: 4.1809e-08 - val_accuracy: 1.0000 - val_f1_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00


In [68]:
deception_pred = deception_model.predict(X_no_deception_train)
deception_pred_test = deception_model.predict(X_no_deception_test)

deception_pred_test_round = deception_pred_test.round()
precision_recall_fscore_support(y_no_deception_test_deception, deception_pred_test_round, average='macro')

#y_test = y_no_deception_test_deception.numpy()
#y_pred = deception_pred_test_round.numpy()

report = classification_report(y_no_deception_test_deception, deception_pred_test_round, output_dict=True)
print(pd.DataFrame(report).transpose())

              precision  recall  f1-score  support
0                   1.0     1.0       1.0    112.0
accuracy            1.0     1.0       1.0      1.0
macro avg           1.0     1.0       1.0    112.0
weighted avg        1.0     1.0       1.0    112.0


In [67]:
y_no_deception_test_deception

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    